<a href="https://colab.research.google.com/github/ShacharYonai/DicatLM-FineTune/blob/main/fine_tune_llm_eedi_kaggle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# !unzip fine-tune-qwen-2.5-eedi.zip -d .
# !rm fine-tune-qwen-2.5-eedi.zip

Archive:  fine-tune-qwen-2.5-eedi.zip
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of fine-tune-qwen-2.5-eedi.zip or
        fine-tune-qwen-2.5-eedi.zip.zip, and cannot find fine-tune-qwen-2.5-eedi.zip.ZIP, period.


In [ ]:
!pip install flash-attn

In [1]:
!pip install bitsandbytes accelerate trl peft

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.6/316.6 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.7/320.7 kB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.8/109.8 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.7/472.7 kB 34.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.4 MB/s eta 0:00:00


In [2]:
!pip install -U sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.8/255.8 kB 18.6 MB/s eta 0:00:00


In [3]:
from sentence_transformers.cross_encoder import CrossEncoder

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [62]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from typing import Callable, Any, Tuple, List
import torch
from torch.utils.data import Dataset, DataLoader
from dataclasses import dataclass
import os
import re
from collections import Counter
from pathlib import Path

In [5]:
from trl import SFTTrainer, DataCollatorForCompletionOnlyLM

In [6]:
from transformers import (AutoTokenizer,
                          AutoModel,
                          DataCollatorWithPadding,
                          AutoTokenizer,
                          AutoModelForCausalLM,
                          BitsAndBytesConfig,
                          DataCollatorForLanguageModeling,
                          TrainingArguments,
                          AutoModelForCausalLM,
                          Trainer,
                          TextStreamer)

In [ ]:
# from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model

In [7]:
misconseption = pd.read_csv("/content/data/misconception_mapping.csv")
sample_data = pd.read_csv("/content/data/sample_submission.csv")
train = pd.read_csv("/content/data/train.csv")
test = pd.read_csv("/content/data/test.csv")

In [8]:
def reshape_wide_to_long(df: pd.DataFrame, id_vars: list[str], value_vars: list[str], var_name: str, value_name: str, sorted_vars: list[str]) -> pd.DataFrame:
    df_reshape = df.melt(id_vars=id_vars, value_vars=value_vars, var_name=var_name, value_name=value_name)
    return df_reshape.sort_values(by=sorted_vars).reset_index(drop=True)

In [9]:
# select the variables for reshaping the misconception
train_tmp_mis = train.loc[:, ["QuestionId","ConstructName","SubjectName","CorrectAnswer","QuestionText",
                              *[var for var in train.columns if var.startswith('Mis')]]]

In [10]:
# select the variables for reshaping the answer
train_tmp_answer = train.loc[:, ["QuestionId","ConstructName","SubjectName","CorrectAnswer","QuestionText",
                              *[var for var in train.columns if var.startswith('Answer')]]]

In [11]:
train_misconception_reshaped = reshape_wide_to_long(df=train_tmp_mis,
                                                    id_vars=["QuestionId","ConstructName","SubjectName","CorrectAnswer","QuestionText"],
                                                    value_vars=[var for var in train.columns if var.startswith('Mis')],
                                                    var_name="misconception",
                                                    value_name="misconception_id",
                                                    sorted_vars=["QuestionId","misconception"])

In [12]:
train_answer_reshaped = reshape_wide_to_long(df=train_tmp_answer,
                                             id_vars=["QuestionId","ConstructName","SubjectName","CorrectAnswer","QuestionText"],
                                             value_vars=[var for var in train.columns if var.startswith('Answer')],
                                             var_name="answer",
                                             value_name="answer_id",
                                             sorted_vars=["QuestionId","answer"])

In [13]:
# concat the two data frames
train_reshaped = pd.concat([train_misconception_reshaped, train_answer_reshaped.loc[:, ["answer","answer_id"]]], axis=1)

In [14]:
# add the misconceptions texts to the reshaped train data
train_reshaped = train_reshaped.merge(misconseption, how="left", left_on="misconception_id", right_on="MisconceptionId")
train_reshaped.drop("MisconceptionId", axis=1, inplace=True)

In [15]:
# drop all the cases where misconception were not assigned
train_reshaped = train_reshaped[~train_reshaped["misconception_id"].isna()].reset_index(drop=True)

In [16]:
train_reshaped.drop(["CorrectAnswer","misconception","misconception_id","answer"], axis=1, inplace=True)

In [17]:
train_reshaped = train_reshaped.assign(txt=lambda x:
                                       "<name>\n" +
                                       x["ConstructName"] +
                                       "\n</name>\n" +
                                       "\n<subject>\n" +
                                       x["SubjectName"] +
                                       "\n</subject>\n" +
                                       "\n<question>\n" +
                                       x["QuestionText"] +
                                       "\n</question>\n" +
                                      "\n<answer>\n" +
                                      x["answer_id"] +
                                       "\n</answer>\n" +
                                      "\n<misconception>\n" +
                                      x["MisconceptionName"] +
                                      "\n</misconception>")

In [18]:
print(train_reshaped["txt"][0])

<name>
Use the order of operations to carry out calculations involving powers
</name>

<subject>
BIDMAS
</subject>

<question>
\[
3 \times 2+4-5
\]
Where do the brackets need to go to make the answer equal \( 13 \) ?
</question>

<answer>
Does not need brackets
</answer>

<misconception>
Confuses the order of operations, believes addition comes before multiplication 
</misconception>


In [18]:
# shufle the data
train_reshaped = train_reshaped.sample(n=train_reshaped.shape[0], replace=False, random_state=42).reset_index(drop=True)
train, valid = train_test_split(train_reshaped, test_size=0.1, random_state=42)

In [46]:
train_prompts = train["txt"].to_list()
valid_prompts = valid["txt"].to_list()

In [ ]:
model_id = "Qwen/Qwen2.5-7B"  # "microsoft/Phi-3.5-mini-instruct"
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token_id = tokenizer.eos_token_id
tokenizer.padding_side = 'right'

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/7.23k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

In [ ]:
# create the Dataset templete
class PromptTemplet(Dataset):
  def __init__(self,
               prompts: list[str],
               tokenizer: AutoTokenizer):
    super().__init__()
    self.prompts = prompts
    self.tokenizer = tokenizer

  def __len__(self) -> int:
    """return the total number of samples"""
    return len(self.prompts)

  def __getitem__(self, index: int):
    """return one sample of image data and label"""
    prompt = self.prompts[index]
    score = tokenizer(prompt, return_tensors='pt')  #  max_length=512, padding='max_length',
    return {
        'prompt_text': prompt,
        'input_ids': score['input_ids'].squeeze(),
        'attention_mask': score['attention_mask'].squeeze()
    }

In [ ]:
train_dataset = PromptTemplet(prompts=train_prompts, tokenizer=tokenizer)
valid_dataset = PromptTemplet(prompts=valid_prompts, tokenizer=tokenizer)

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


In [ ]:
bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16,
        bnb_4bit_use_double_quant=True,
        llm_int8_enable_fp32_cpu_offload=True,
    )

In [ ]:
lora_config = LoraConfig(
        r=16,
        lora_alpha=8,
        target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
        lora_dropout=0.05,
        bias="none",
        task_type="CAUSAL_LM"
    )

In [ ]:
model = AutoModelForCausalLM.from_pretrained(model_id,
                                             quantization_config=bnb_config,
                                             device_map="auto",
                                             trust_remote_code=True)

config.json:   0%|          | 0.00/686 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/27.8k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/3.95G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/3.56G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/138 [00:00<?, ?B/s]

In [ ]:
model = prepare_model_for_kbit_training(model, use_gradient_checkpointing=True)

In [ ]:
model = get_peft_model(model, lora_config)

In [ ]:
model.config.pad_token_id = tokenizer.eos_token_id

In [ ]:
trainable, total = model.get_nb_trainable_parameters()
print(f"Trainable: {trainable} | total: {total} | Percentage: {trainable/total*100:.4f}%")

Trainable: 10092544 | total: 7625709056 | Percentage: 0.1323%


In [ ]:
@dataclass
class TrainModel:
    output_dir: str
    run_name: str
    trainer_type: Callable
    model: Any
    tokenizer: Any
    train_data: Dataset
    valid_data: Dataset
    overwrite_output_dir: bool = True
    learning_rate: float = 0.00015  # 0.000015
    auto_find_batch_size: bool = True
    per_device_train_batch_size: int =32  # 16
    per_device_eval_batch_size: int = 16
    gradient_accumulation_steps: int = 2
    gradient_checkpointing: bool = True
    eval_accumulation_steps = 10
    warmup_steps: int = 20
    save_total_limit: int = 5
    num_train_epochs: int = 2
    weight_decay: float = 0.0001
    optim: str = "adamw_torch"  # "adamw_torch"  # "adamw_torch" 'lion_32bit'
    logging_steps = 1
    fp16: bool = True
    bf16: bool = True
    tf32: bool = True
    max_grad_norm: float = 0.3
    warmup_ratio: float = 0.03
    lr_scheduler_type: str = "cosine"
    eval_strategy: str = "steps"  # "epoch"
    save_strategy: str = "steps"  # "epoch"
    load_best_model_at_end: bool = True
    dataset_text_field: str = "prompt_text"
    metric_for_best_model: str = 'loss'
    push_to_hub: bool = False
    report_to = "wandb"

    @property
    def data_collator(self):
        return DataCollatorForLanguageModeling(self.tokenizer, mlm=False)

    @property
    def training_args(self):
        return TrainingArguments(
            output_dir=self.output_dir,
            run_name=self.run_name,
            overwrite_output_dir=self.overwrite_output_dir,
            learning_rate=self.learning_rate,
            gradient_accumulation_steps=self.gradient_accumulation_steps,
            auto_find_batch_size=self.auto_find_batch_size,
            per_device_train_batch_size=self.per_device_train_batch_size,
            per_device_eval_batch_size=self.per_device_eval_batch_size,
            eval_accumulation_steps=self.eval_accumulation_steps,
            # gradient_checkpointing=self.gradient_checkpointing,
            warmup_steps=self.warmup_steps,
            save_total_limit=self.save_total_limit,
            num_train_epochs=self.num_train_epochs,
            weight_decay=self.weight_decay,
            optim=self.optim,
            logging_steps=self.logging_steps,
            fp16=self.fp16,
            # bf16=self.bf16,
            # tf32=self.tf32,
            # max_grad_norm=self.max_grad_norm,
            # warmup_ratio=self.warmup_ratio,
            lr_scheduler_type=self.lr_scheduler_type,
            eval_strategy=self.eval_strategy,
            save_strategy=self.save_strategy,
            metric_for_best_model=self.metric_for_best_model,
            load_best_model_at_end=self.load_best_model_at_end,
            push_to_hub=self.push_to_hub,
            #report_to=self.report_to
        )

    @property
    def train(self):
        trainer_obj = SFTTrainer(
            model=self.model,
            args=self.training_args,
            train_dataset=self.train_data,
            eval_dataset=self.valid_data,
            tokenizer=self.tokenizer,
            data_collator=self.data_collator,
        )
        return trainer_obj.train()

In [ ]:
torch.cuda.empty_cache()

In [ ]:
trainer = TrainModel(output_dir="fine_tune_qwen_2.5",
                     run_name="fine_tunning_qwen_2.5_eedi",
                     trainer_type=Trainer,
                     model=model,
                     tokenizer=tokenizer,
                     train_data=train_dataset,
                     valid_data=valid_dataset,
                    )

In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:292: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]


Step,Training Loss,Validation Loss
1,1.900500,1.881471
2,1.954200,1.881303
3,1.901500,1.880659
4,1.863300,1.879346
5,1.914000,1.876922
6,1.849700,1.873215
7,1.926200,1.868224
8,1.963500,1.862104
9,1.851500,1.854452
10,1.913100,1.844866


We detected that you are passing `past_key_values` as a tuple and this is deprecated and will be removed in v4.43. Please use an appropriate `Cache` class (https://huggingface.co/docs/transformers/v4.41.3/en/internal/generation_utils#transformers.Cache)


Step,Training Loss,Validation Loss
1,1.224500,1.234120
2,1.220500,1.232921
3,1.228000,1.230395
4,1.202000,1.226397
5,1.252200,1.221235
6,1.259600,1.214796
7,1.166900,1.207389
8,1.255600,1.199196
9,1.248500,1.190207
10,1.123500,1.180769


TypeError: 'TrainOutput' object is not callable

In [ ]:
trainer.model.save_pretrained("fine-tune-qwen-2.5-eedi")

In [21]:
torch.cuda.empty_cache()

In [20]:
bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16,
        bnb_4bit_use_double_quant=True,
        llm_int8_enable_fp32_cpu_offload=True,
    )

In [21]:
base_model = "Qwen/Qwen2.5-7B"  # "microsoft/Phi-3.5-mini-instruct"
fine_tuned_model = "/content/fine-tune-qwen-2.5-eedi"  # "/content/fine-tune-phi-3.5-mini-instruct-eedi"

In [22]:
model_base = AutoModelForCausalLM.from_pretrained(base_model, quantization_config=bnb_config, return_dict=True, device_map='auto')
# model_fine_tuned = AutoModelForCausalLM.from_pretrained(fine_tuned_model, quantization_config=bnb_config, return_dict=True, device_map='auto')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/686 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/27.8k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/3.95G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/3.56G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/138 [00:00<?, ?B/s]

In [47]:
# del model_fine_tuned
import gc
gc.collect()
torch.cuda.empty_cache()

In [24]:
tokenizer = AutoTokenizer.from_pretrained(base_model)

tokenizer_config.json:   0%|          | 0.00/7.23k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

In [25]:
# next we'll add our trained adapter to the model
model_base.load_adapter(fine_tuned_model, adapter_name="adapter")

In [26]:
# now using enable_adapters and disable_adapters we can choose
# if we want to run inference on the model itself or have it be
# influenced by our newly trained weights
model_base.enable_adapters()
# model.disable_adapters()

In [27]:
# also make sure we set the pad token, and for good measure turn off caching
model_base.config.pad_token_id = tokenizer.pad_token_id
model_base.config.use_cache = False
model_base.config.pad_token_id = tokenizer.eos_token_id

In [ ]:
# model_fine_tuned.config.pad_token_id = tokenizer.pad_token_id
# model_fine_tuned.config.use_cache = False
# model_fine_tuned.config.pad_token_id = tokenizer.eos_token_id

In [28]:
model_base.eval()

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(152064, 3584)
    (layers): ModuleList(
      (0-27): 28 x Qwen2DecoderLayer(
        (self_attn): Qwen2SdpaAttention(
          (q_proj): lora.Linear4bit(
            (base_layer): Linear4bit(in_features=3584, out_features=3584, bias=True)
            (lora_dropout): ModuleDict(
              (adapter): Dropout(p=0.05, inplace=False)
            )
            (lora_A): ModuleDict(
              (adapter): Linear(in_features=3584, out_features=16, bias=False)
            )
            (lora_B): ModuleDict(
              (adapter): Linear(in_features=16, out_features=3584, bias=False)
            )
            (lora_embedding_A): ParameterDict()
            (lora_embedding_B): ParameterDict()
            (lora_magnitude_vector): ModuleDict()
          )
          (k_proj): lora.Linear4bit(
            (base_layer): Linear4bit(in_features=3584, out_features=512, bias=True)
            (lora_dropout): ModuleDict(
     

In [ ]:
# model_fine_tuned.eval()

Phi3ForCausalLM(
  (model): Phi3Model(
    (embed_tokens): Embedding(32064, 3072, padding_idx=32000)
    (embed_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-31): 32 x Phi3DecoderLayer(
        (self_attn): Phi3Attention(
          (o_proj): lora.Linear4bit(
            (base_layer): Linear4bit(in_features=3072, out_features=3072, bias=False)
            (lora_dropout): ModuleDict(
              (default): Dropout(p=0.05, inplace=False)
            )
            (lora_A): ModuleDict(
              (default): Linear(in_features=3072, out_features=16, bias=False)
            )
            (lora_B): ModuleDict(
              (default): Linear(in_features=16, out_features=3072, bias=False)
            )
            (lora_embedding_A): ParameterDict()
            (lora_embedding_B): ParameterDict()
            (lora_magnitude_vector): ModuleDict()
          )
          (qkv_proj): Linear4bit(in_features=3072, out_features=9216, bias=False)
          (rotary_emb)

In [ ]:
len(valid_prompts)

437

In [29]:
# Load a pretrained CrossEncoder model
cross_encoder_model = CrossEncoder("cross-encoder/ms-marco-MiniLM-L-12-v2", default_activation_function=torch.nn.Sigmoid())
cross_encoder_model.to(device)

config.json:   0%|          | 0.00/791 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/134M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [30]:
list_of_misconseption = misconseption["MisconceptionName"].to_list()

In [31]:
def create_misconception_prompt(prompt: str) -> str:
  txt_to_delete = re.findall("<misconception>\n(.*\n</misconception>)", prompt)[0]
  p=re.sub(txt_to_delete, "", prompt)
  return p

In [130]:
def generate_misconception(misconception_prompt: str,
                           tokenizer: AutoTokenizer,
                           temperature: float = 0.7,
                           max_new_tokens: int = 30,
                           num_retries: int = 5) -> list[str]:
  device = "cuda" if torch.cuda.is_available() else "cpu"
  streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)
  inputs = tokenizer(misconception_prompt, return_tensors="pt").to(device)
  predicted_misconceptions = []
  for _ in range(num_retries):
    generated_ids_model_base = model_base.generate(**inputs,
                                                  temperature=temperature,
                                                  # streamer=streamer,
                                                  max_new_tokens=max_new_tokens,
                                                  do_sample=True,
                                                  pad_token_id=tokenizer.eos_token_id)
    decoded_model_base = tokenizer.batch_decode(generated_ids_model_base)
    try:
      predicted_misconception = re.findall("<mis.*>\n(.*)\n</mis.*>?", decoded_model_base[0])[0]
      predicted_misconceptions.append(predicted_misconception)
    except Exception as e:
      print(f"Error: {e}")
      predicted_misconceptions.append(None)
  return predicted_misconceptions

In [33]:
def extract_similar_misconception(predicted_misconceptions: str, list_of_misconseption: list[str], cross_encoder_model: Any) -> list[int]:
  """
  the functions recieves at most three generated misconceptions.
  It then generated list of three ranked similar misconceptions.
  I aggregate the misconceptions into a single list and rank the misconceptions by their frequency
  in descending order
  """
  general_corpuses = []
  for misconception in predicted_misconceptions:
    ranks = cross_encoder_model.rank(misconception, list_of_misconseption)
    list_of_corpuses = []
    for corpus in ranks[:100]:
      list_of_corpuses.append(corpus["corpus_id"])
    general_corpuses.extend(list_of_corpuses)
  misconceptions_frequencies = Counter(general_corpuses).most_common()
  top_misconceptions = [id for id, _ in misconceptions_frequencies[:25]]
  return top_misconceptions


In [34]:
number=20
misconception_prompt = create_misconception_prompt(valid_prompts[number])

In [36]:
predicted_misconceptions = generate_misconception(misconception_prompt, tokenizer)

In [37]:
similar_misconception = extract_similar_misconception(predicted_misconceptions, list_of_misconseption, cross_encoder_model)

In [38]:
print(similar_misconception)

[47, 1412, 1303, 1654, 594, 1475, 2465, 1949, 1541, 283, 101, 1584, 563, 2014, 2380, 422, 1889, 362, 2497, 1234, 1312, 165, 1893, 866, 2083]


In [39]:
original_misconception = re.findall("<misconception>\n(.*)\n</misconception>", valid_prompts[number])[0]
print(original_misconception)

Believes y=-f(x) is a reflection in y=x


In [40]:
# locate the index place of the original misconception in the list
index_of_original_misconception = [i for i, x in enumerate(list_of_misconseption) if x == original_misconception]
print(index_of_original_misconception)

[47]


#### *In* order to mimic the process of running the fine-tune model on the entire valid data on the gpu I need to drop the misconception and create a new data without it

In [48]:
prompts_for_gpu = []
for prompt in valid_prompts:
  prompts_for_gpu.append(create_misconception_prompt(prompt))

In [137]:
# create the Dataset templete
class PromptTemplet(Dataset):
  def __init__(self,
               prompts: list[str],
               tokenizer: AutoTokenizer,
               base_model: Any,
               cross_encoder_model: Any,
               list_of_misconseption: list[str],
               num_retries: int = 5,
               temperature: float = 0.7,
               max_new_tokens: int = 30,
               device: str = "cuda" if torch.cuda.is_available() else "cpu"
               ):
    super().__init__()
    self.prompts = prompts
    self.tokenizer = tokenizer
    self.base_model = base_model
    self.cross_encoder_model = cross_encoder_model
    self.list_of_misconseption = list_of_misconseption
    self.num_retries = num_retries
    self.temperature = temperature
    self.max_new_tokens = max_new_tokens
    self.device = device

  def predict_misconceptions(self, input: str):
    device = "cuda" if torch.cuda.is_available() else "cpu"
    # streamer = TextStreamer(self.tokenizer, skip_prompt=True, skip_special_tokens=True)
    predicted_misconceptions = []
    for _ in range(self.num_retries):
      generated_ids_base_model = self.base_model.generate(**input,
                                                          temperature=self.temperature,
                                                          # streamer=streamer,
                                                          max_new_tokens=self.max_new_tokens,
                                                          do_sample=True,
                                                          pad_token_id=tokenizer.eos_token_id)
      decoded_model_base = tokenizer.batch_decode(generated_ids_base_model)
      try:
        predicted_misconception = re.findall("<mis.*>\n(.*)\n</mis.*>?", decoded_model_base[0])[0]
        predicted_misconceptions.append(predicted_misconception)
      except Exception as e:
        print(f"Error: {e}")
        predicted_misconceptions.append(None)
    return predicted_misconceptions

  def extract_similar_misconception(self,
                                    predicted_misconceptions: list[str],
                                    list_of_misconseption: list[str],
                                    cross_encoder_model: Any) -> list[int]:
    """
    the functions recieves at most three generated misconceptions.
    It then generated list of three ranked similar misconceptions.
    I aggregate the misconceptions into a single list and rank the misconceptions by their frequency
    in descending order
    """
    general_corpuses = []
    for misconception in predicted_misconceptions:
      ranks = cross_encoder_model.rank(misconception, list_of_misconseption)
      list_of_corpuses = []
      for corpus in ranks[:100]:
        list_of_corpuses.append(corpus["corpus_id"])
      general_corpuses.extend(list_of_corpuses)
    misconceptions_frequencies = Counter(general_corpuses).most_common()
    top_misconceptions = [id for id, _ in misconceptions_frequencies[:25]]
    return top_misconceptions

  def __len__(self) -> int:
    """return the total number of samples"""
    return len(self.prompts)

  def __getitem__(self, index: int):
    """return one sample of image data and label"""
    prompt = self.prompts[index]
    input = tokenizer(prompt, return_tensors='pt').to(self.device)
    generated_misconceptions = self.predict_misconceptions(input)
    similar_misconception = self.extract_similar_misconception(generated_misconceptions, self.list_of_misconseption, self.cross_encoder_model)
    return similar_misconception

In [138]:
valid_dataset_gpu = PromptTemplet(prompts=prompts_for_gpu,
                                  tokenizer=tokenizer,
                                  base_model=model_base,
                                  cross_encoder_model=cross_encoder_model,
                                  list_of_misconseption=list_of_misconseption)

In [139]:
valid_dataloader_gpu = DataLoader(valid_dataset_gpu, batch_size=2)

In [140]:
for batch in valid_dataloader_gpu:
  break

In [141]:
print(batch)

[tensor([ 570, 1975]), tensor([2137, 1337]), tensor([ 773, 1765]), tensor([1791,  340]), tensor([1959, 1017]), tensor([2177,   14]), tensor([545, 606]), tensor([1815, 1708]), tensor([465, 170]), tensor([2334,  888]), tensor([ 367, 2112]), tensor([725,  79]), tensor([2236, 1206]), tensor([1231, 2292]), tensor([   0, 1316]), tensor([1527,  336]), tensor([2341, 1450]), tensor([1180,  282]), tensor([ 18, 931]), tensor([2394, 1338]), tensor([ 515, 2263]), tensor([449, 862]), tensor([1746,  120]), tensor([2511, 1929]), tensor([ 686, 1224])]


In [142]:
torch.stack(batch, axis=1)

tensor([[ 570, 2137,  773, 1791, 1959, 2177,  545, 1815,  465, 2334,  367,  725,
         2236, 1231,    0, 1527, 2341, 1180,   18, 2394,  515,  449, 1746, 2511,
          686],
        [1975, 1337, 1765,  340, 1017,   14,  606, 1708,  170,  888, 2112,   79,
         1206, 2292, 1316,  336, 1450,  282,  931, 1338, 2263,  862,  120, 1929,
         1224]])